In [1]:
import tomica
print(tomica.__file__)

/Users/zhangyangfa/rust-projects/tomica/tomica/__init__.py


In [2]:
import polars as pl

In [3]:
data = pl.Series([-1.0,-2.0,-3.0,3.0,4.5,5.0,0,2.3,4.4,5.5,6.6])
data

""
f64
-1.0
-2.0
-3.0
3.0
4.5
…
0.0
2.3
4.4


In [4]:
tomica.operators.ts_delay(data,1)

""
f64
null
-1.0
-2.0
-3.0
3.0
…
5.0
0.0
2.3


In [5]:
tomica.operators.ts_delta(data,1)

""
f64
null
-1.0
-1.0
6.0
1.5
…
-5.0
2.3
2.1


In [6]:
tomica.operators.ts_returns(data,1)

""
f64
null
1.0
0.5
-2.0
0.5
…
-1.0
inf
0.913043


In [7]:
tomica.operators.ts_sum(data,1)

""
f64
-1.0
-2.0
-3.0
3.0
4.5
…
0.0
2.3
4.4


In [8]:
tomica.operators.ts_mean(data,3)

""
f64
-1.0
-1.5
-2.0
-0.666667
1.5
…
3.166667
2.433333
2.233333


In [9]:
tomica.operators.ts_max(data,3)

""
f64
-1.0
-1.0
-1.0
3.0
4.5
…
5.0
5.0
4.4


In [10]:
tomica.operators.ts_min(data,3)

""
f64
-1.0
-2.0
-3.0
-3.0
-3.0
…
0.0
0.0
0.0


In [11]:
tomica.operators.ts_std(data,3)

""
f64
null
0.707107
1.0
3.21455
3.968627
…
2.753785
2.502665
2.200757


In [12]:
tomica.operators.ts_rank(data,3)


thread '<unnamed>' (77464271) panicked at /Users/zhangyangfa/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/polars-compute-0.52.0/src/rolling/no_nulls/rank.rs:110:9:
internal error: entered unreachable code: expected RollingFnParams::Rank
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: internal error: entered unreachable code: expected RollingFnParams::Rank

In [13]:
tomica.operators.ts_variance(data,3)

""
f64
null
0.5
1.0
10.333333
15.75
…
7.583333
6.263333
4.843333
